In [1]:
import os
%pwd
os.chdir('../')
%pwd

'c:\\Projects\\FinSight'

In [2]:
# src/finance_ml/entity/config_entity.py

from dataclasses import dataclass

@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: str
    raw_data_file: str
    period: str
    interval: str

In [3]:
# src/finance_ml/config/configuration.py

from finance_ml.constants import *
from finance_ml.utils.common import read_yaml, create_directories
from finance_ml.entity.config_entity import DataIngestionConfig
import os

class ConfigurationManager:
    def __init__(
        self,
        config_filepath=CONFIG_FILE_PATH,
        params_filepath=PARAMS_FILE_PATH,
        schema_filepath=SCHEMA_FILE_PATH
    ):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])

    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion
        params = self.params.data_ingestion # Access data_ingestion params from params.yaml

        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            raw_data_file=os.path.join(config.root_dir, config.raw_data_file), 
            period=params.period,     
            interval=params.interval  
        )

        return data_ingestion_config

In [4]:
#src/finance_ml/components/data_ingestion.py
import yfinance as yf
import pandas as pd
from src.finance_ml import logger
from src.finance_ml.entity.config_entity import DataIngestionConfig

class DataIngestion:
    def __init__(self, config: DataIngestionConfig, ticker: str):
        self.config = config
        self.ticker = ticker

    def download_data(self):
        logger.info(f"Downloading data for {self.ticker}")
        df = yf.download(
            tickers=self.ticker,
            period=self.config.period,
            interval=self.config.interval
        )
        df.to_csv(self.config.raw_data_file)
        logger.info(f"Data saved to {self.config.raw_data_file}")


In [5]:
#src/finance_ml/pipeline/stage_01_data_ingestion.py

from src.finance_ml.config.configuration import ConfigurationManager
from src.finance_ml.components.data_ingestion import DataIngestion
from src.finance_ml import logger

STAGE_NAME = "Data Ingestion stage"

class DataIngestionTrainingPipeline:
    def __init__(self, ticker: str):
        self.ticker = ticker

    def main(self):
        config = ConfigurationManager()
        data_config = config.get_data_ingestion_config()

        data_ingestor = DataIngestion(config=data_config, ticker=self.ticker)
        data_ingestor.download_data()
